# Exercise Sheet 02: Principal Component Analysis, Factor Analysis, K-Means, and Gaussian Mixture Models

**Introduction to Data Mining WS24/25**  
**Bielefeld University**  
**Alina Deriyeva, Adia Khalid, Benjamin Paaßen**  
**Exercise Sheet Publication Date: 2024-11-04**  
**Exercise Sheet Submission Deadline: Friday, 2024-11-15, noon (i.e. 12:00), via moodle**

**NOTE** The use of language models/AI tools is permitted under three conditions
1. transparency: you tell us that you used them
2. accountability: you take full responsibility for the submission, can explain and defend it
3. privacy: you do not transmit any private information to any external tool

We also appreciate it if you link to a chatlog of the interaction with the language model/AI tool for research purposes.

### Task 02.01

Assume we have some encoding function $\phi : R^n \to R$ which maps input data points to single numbers in the latent space. Further, assume we want to find a linear decoding function $\psi(z) = w \cdot z$ which maps numbers in the latent space back to the input space such that the squared reconstruction error for an input data set $x_1, \ldots, x_N \in R^n$ is minimized.

Formalize this problem as a minimization problem.

**ANSWER:** PLEASE PROVIDE LATEX CODE OR AN IMAGE OF YOUR DERIVATION HERE

### Task 02.02

Recall the equation for the expected negative log likelihood in a Gaussian mixture model from the lecture:

\begin{align*}
Q = &\sum_{i=1}^N \sum_{k=1}^K -\gamma_{k,i} \cdot \log\Big[ p_{X|Z}(x_i|k) \cdot p_Z(k) \Big]\\
=& \sum_{i=1}^N \sum_{k=1}^K \gamma_{k,i} \cdot \Big(\frac{1}{2}\log[2\pi \det(\Sigma_k)] + \frac{1}{2} (x_i - \mu_k)^T \Sigma_k^{-1} (x_i - \mu_k) - \log[p_Z(k)]\Big)
\end{align*}

Assuming that $Q$ is convex, find the optimal value for $\mu_k$.

**HINT:** You may use the following general matrix/vector gradient equation (refer to the [matrix cook book by Peterson and Pedersen (2012), p.10-11](https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf) :
\begin{align*}
\nabla_x (x - y)^T W (x - y) &= 2 W (x-y)
\end{align*}

**ANSWER:** PLEASE PROVIDE LATEX CODE OR AN IMAGE OF YOUR DERIVATION HERE

## Preamble: Data set

The file `sheet02_data.csv` contains fictional data as you might find in an online course. Each row represents a student, each column a feature of the student's activity in the course, namely their number of posts in the course discussion forum, the number of questions they asked in chat during the online lectures, the number of messages they sent to their peers, and the number of points they achieved in each of the five exercises of the course.

Note that there is quite a bit of missing data for later exercises because many students dropped out of the course early.

The following code loads this raw data and prints it.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

features = ['num_forum_postings',
    'num_questions',
    'num_messages',
    'num_completed_tasks',
    'points_exercise_1',
    'points_exercise_2',
    'points_exercise_3',
    'points_exercise_4',
    'points_exercise_5']

X = np.loadtxt('sheet02_data.csv', skiprows = 1, delimiter = '\t')
print(X)

[[ 3.  1.  1. ... 56. 61. 60.]
 [ 0.  0.  0. ... nan nan nan]
 [ 7.  3.  2. ... 66. 57. nan]
 ...
 [ 0.  0.  0. ... 30. nan nan]
 [ 1.  0.  0. ... nan nan nan]
 [ 3.  0.  1. ... 40. 35. 40.]]


### Task 02.03

Our first challenge is to impute the missing data. Fill in missing values with the mean points the respective student got on the other exercises. For students with no completed exercises, fill in zeros.

### Task 02.04

Next, normalize the data by dividing by the maximum value in each column.

## Principal Component Analysis

### Task 02.05

Compute the covariance matrix of the data via `np.cov` and compute the eigenvalues of the covariance matrix via `np.linalg.eigvals`. Provide a plot of the eigenvalues on the y-axis, sorted according to size (the largest eigenvalue at x=0, the second-largest on x=1, and so on).

Compute and report the percentage of variance covered by the first two eigenvalues.

**HINT:** `np.cov` treats the rows as variables and columns as observations. For our data set, rows are observations and columns are variables.

### Task 02.06

Use the `fit` method of a `sklearn.decomposition.PCA` model to perform a principal component analysis of this data with `n_components = 2`.

Transform the data to the latent space via the `transform` function of the PCA model.

Plot the data using a 2D scatter plot.

In [2]:
from sklearn.decomposition import PCA


## Factor Analysis

### Task 02.07

Use the `fit` method of a `sklearn.decomposition.FactorAnalysis` model to perform a factor analysis of this data with `n_components = 2`. Use the `rotation = 'varimax'` parameter.

Transform the data to the latent space via the `transform` function of the FA model.

Plot the data using a 2D scatter plot.

Compare this plot to the plot above. What difference do you notice?

In [3]:
from sklearn.decomposition import FactorAnalysis

**ANSWER:**

### Task 02.08

Print the factors found by the factor analysis using `print(model.components_)`. Try to interpret both factors. What does the first factor represent? What does the second factor represent?

**ANSWER:**

## K-Means Clustering

### Task 02.09

Using `sklearn.cluster.KMeans`, perform cluster analyses of the data for `n_clusters` between 2 and 10. For each value of `n_clusters`, compute the `sklearn.metrics.silhouette_score`. Provide a plot of the silhouette score on the y axis and `n_clusters` on the x axis. Report which value for `n_clusters` is best according to this analysis.

**HINT:** The `silhouette_score` function requires the cluster labels as second argument. You can retrieve the cluster labels from a fitted `KMeans` model via the `predict` function.

In [4]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

### Task 02.10

Using `sklearn.cluster.KMeans`, perform a cluster analysis of the data with `n_clusters = 2`.

Get the cluster membership of each point via the `predict` function of the KMeans model.

Provide a scatter plot of the latent representation of the data according to factor analysis, where the color of each point represents the cluster membership.

## Gaussian Mixture Models

### Task 02.11

Using `sklearn.mixture.GaussianMixture`, perform cluster analyses of the data with `n_components` between 2 and 10. For each cluster analysis, compute the `bic` function value of the model (this is the Bayesian information criterion). Provide a plot of the bic value on the y axis with `n_components` on the x axis.

Report which value for `n_components` is best according to this analysis.

In [5]:
from sklearn.mixture import GaussianMixture

**ANSWER:**

### Task 02.12

Using `sklearn.mixture.GaussianMixture`, perform a cluster analysis of the data with `n_components = 2`.

Get the cluster membership of each point via the `predict` function of the GaussianMixture model.

Provide a scatter plot of the latent representation of the data according to factor analysis, where the color of each point represents the cluster membership.

### Task 02.13

Using `sklearn.mixture.GaussianMixture`, perform a cluster analysis of the latent space representation according to factor analysis with `n_components = 3`.

Get the cluster membership of each point via the `predict` function of the GaussianMixture model.

Provide a scatter plot, where the color of each point represents the cluster membership.

### Task 02.14

Print the mean feature values for each cluster. Try to interpret the clusters: What characterizes the differenct clusters?

**ANSWER:**